In [1]:
%matplotlib notebook

In [2]:
!nvidia-smi


Thu Aug 19 10:47:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      Off  | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    34W / 250W |   2256MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-PCIE-40GB      Off  | 00000000:25:00.0 Off |                    0 |
| N/A   

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd

# Python 3 standard library
from pathlib import Path

from model.collectdata_kde_Ellipsoids import collect_t2kde_data

##  It takes 9 input features (poca centers + (A,B,C,D,E,F)) . 
from model.models_kde import TracksToKDE_Ellipsoids_DDplus as Model

# kde_loss_Ba is a Chi2 loss (go and look at it!)
from model.kde_loss_Ba import Loss

from model.training_kde import validate, select_gpu, Results
from model.plots import dual_train_plots, dual_train_plots_tracks_to_KDE, replace_in_ax

In [7]:
folder = '11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06'
previous = 'jgo_files/28July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_it29pretrain2_250_epochs_1e-05/28July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_it29pretrain2_250_epochs_1e-05_final.pyt'
arch = [5, 7, 10, 25, 50, 75, 100, 150, 250, 350, 500]
batch_size = 64
latentChannels = 8

device = select_gpu(1)


1 available GPUs (initially using device 0):
  0 A100-PCIE-40GB


In [5]:
# Data Import. If we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
# the dataset will overflow the GPU memory; device=device will allow the data to move back
# and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
# down performance by about 10%.  So comment out when not needed.

## Training dataset. You can put as many files here as desired.

train_loader = collect_t2kde_data('/share/lazy/will/data/June30_2020_80k_1.h5', 
                                  '/share/lazy/will/data/June30_2020_80k_2.h5',
                                    '/share/lazy/will/data/June30_2020_80k_3.h5',
                                    '/share/lazy/will/data/June30_2020_80k_4.h5',
# #                                       '/share/lazy/will/data/June30_2020_80k_5.h5',
#                                       #'/share/lazy/will/data/June30_2020_80k_6.h5',
#                                       #'/share/lazy/will/data/June30_2020_80k_7.h5',
                             batch_size=batch_size,
## if we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
## the dataset will overflow the GPU memory; device=device will allow the data to move back
## and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
## down performance by about 10%.  So comment out when not needed.
#                               device=device,
                           # slice = slice(None,10000)
                           )

# Validation dataset. You can slice to reduce the size.
## mds no separate validation set yet,

## For iter12, change slice(18000,None) to slice(10000,None)
## First, we'll see if this changes the validation cost significantly
##  Second, we will see if this reduces the validation cost fluctuations
val_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                            batch_size=batch_size,
#                                 device=device,
                             #slice = slice(10000,None)
                           )

Loading data...


/data/home/jgocain/.local/lib/python3.8/site-packages/awkward0/persist.py:509: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  schema = schema.tostring()


pocaMx.shape =  (80000,)
nEvts =  80000
len(pocaMx[0]) =  199
len(pocaMx[1]) =  25
len(pocaMx[2]) =  369
len(pocaMx[3]) =  143
len(pocaMx[4]) =  160
majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 199
 iEvt, nTrks =  1 25
 iEvt, nTrks =  2 369
 iEvt, nTrks =  3 143
 iEvt, nTrks =  4 160
 iEvt, nTrks =  5 260
 iEvt, nTrks =  6 237
 iEvt, nTrks =  7 327
 iEvt, nTrks =  8 178
 iEvt, nTrks =  9 106
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  0.00045611936
majorAxis[iTrk][1][0] =  -4.8292455e-05
majorAxis[iTrk][2][0] =  0.090019904
minorAxis_1[iTrk][0][0] =  -1.8602173
minorAxis_1[iTrk][1][0] =  -17.569641
minorAxis_1[iTrk][2][0] =  4.7891795e-08
minorAxis_2[iTrk][0][0] =  -17.569414
minorAxis_2[iTrk][1][0] =  1.8601931
minorAxis_2[iTrk][2][0] =  0.0900199
  
majorAxis[iTrk][0][0] =  0.002360258
majorAxis[iTrk][1][0] =  -0.007426616
majorAxis[iTrk][2][0] =  0.3710108


majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 196
 iEvt, nTrks =  1 35
 iEvt, nTrks =  2 265
 iEvt, nTrks =  3 128
 iEvt, nTrks =  4 124
 iEvt, nTrks =  5 122
 iEvt, nTrks =  6 300
 iEvt, nTrks =  7 179
 iEvt, nTrks =  8 243
 iEvt, nTrks =  9 112
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  -2.440992e-05
majorAxis[iTrk][1][0] =  -0.00038087237
majorAxis[iTrk][2][0] =  0.08211532
minorAxis_1[iTrk][0][0] =  -17.631672
minorAxis_1[iTrk][1][0] =  1.130005
minorAxis_1[iTrk][2][0] =  4.7444193e-08
minorAxis_2[iTrk][0][0] =  1.1299927
minorAxis_2[iTrk][1][0] =  17.631481
minorAxis_2[iTrk][2][0] =  0.08211532
  
majorAxis[iTrk][0][0] =  -0.02277395
majorAxis[iTrk][1][0] =  0.0014473597
majorAxis[iTrk][2][0] =  0.6347583
minorAxis_1[iTrk][0][0] =  -1.1205891
minorAxis_1[iTrk][1][0] =  -17.632273
minorAxis_1[iTrk][2][0] =  2.1543272e-09
minorAxis_2[iTrk][0][0] =  -17.6208

Loading data...
pocaMx.shape =  (20000,)
nEvts =  20000
len(pocaMx[0]) =  211
len(pocaMx[1]) =  21
len(pocaMx[2]) =  20
len(pocaMx[3]) =  198
len(pocaMx[4]) =  233
majorAxis.shape =  (20000, 3)
minorAxis_1.shape =  (20000, 3)
minorAxis_2.shape =  (20000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  20000
 iEvt, nTrks =  0 211
 iEvt, nTrks =  1 21
 iEvt, nTrks =  2 20
 iEvt, nTrks =  3 198
 iEvt, nTrks =  4 233
 iEvt, nTrks =  5 85
 iEvt, nTrks =  6 223
 iEvt, nTrks =  7 425
 iEvt, nTrks =  8 252
 iEvt, nTrks =  9 169
A.shape =  (20000,)
majorAxis[iTrk][0][0] =  -0.00023452607
majorAxis[iTrk][1][0] =  -0.00047206535
majorAxis[iTrk][2][0] =  0.096502915
minorAxis_1[iTrk][0][0] =  -15.822749
minorAxis_1[iTrk][1][0] =  7.8608756
minorAxis_1[iTrk][2][0] =  -2.6228399e-08
minorAxis_2[iTrk][0][0] =  7.860759
minorAxis_2[iTrk][1][0] =  15.822513
minorAxis_2[iTrk][2][0] =  0.096502915
  
majorAxis[iTrk][0][0] =  -0.37655562
majorAxis[iTrk][1][0] =  0.2768704
majorAxis[iTrk][2][0] = 

In [8]:
# This gets built up during the run - do not rerun this cell
results = pd.DataFrame([], columns=Results._fields)
nOut1, nOut2, nOut3, nOut4, nOut5, nOut6, nOut7, nOut8, nOut9, nOut10, nOut11 = arch
model = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11)

batch_size = 64
loss = Loss(epsilon=3e-6)

epochs = folder[folder.index('epochs')-4:folder.index('epochs')-1]
if '_' in epochs:
    epochs = epochs[1:]
epochs = range(int(epochs))

for epoch in [0]:
    suffix = str(41)
    dict_name = 'jgo_files/' + folder + '/' + folder + '_'+ suffix + '.pyt'
    print('Currently Analyzing', dict_name)
    model_dict = model.state_dict()
    model = model.to(device)
    index = 0
    for k,v in model_dict.items():
        #print("index, k =  ",index,"  ",k)
        index = index+1
    try:
        updated_dict = model_dict
        model.load_state_dict(updated_dict,strict=False)
        model_dict = model.state_dict()
        pretrained_dict = torch.load(dict_name)

        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        #print("pretrained_dict iterated")
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        ##
        #   when starting from a model with a fully connected last layer rather than a convolutional layer
        # 3. load the new state dict
        #   need to use strict=False as the two models state model attributes do not agree exactly
        #   see https://pytorch.org/docs/master/_modules/torch/nn/modules/module.html#Module.load_state_dict
        model.load_state_dict(pretrained_dict,strict=False)

        with torch.no_grad():
            valcosts = 0
            print('Starting Test Set')
            val_start = time.time()
            for inputs, labels in val_loader:
                if inputs.device != device:
                    inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                costs = loss(outputs,labels)
                valcosts += costs.data.item()
                #print(costs.data.item())
            print('Completed in ' + str(time.time() - val_start) + ' s')
            traincosts = 0
            train_start = time.time()
            print('Starting Train Set')
            for inputs, labels in train_loader:
                if inputs.device != device:
                    inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                costs = loss(outputs,labels)
                traincosts += costs.data.item()
                #print(costs.data.item())
            print('Completed in ' + str(time.time() - train_start) + ' s')
            holding = pd.DataFrame([[epoch, traincosts/len(train_loader), valcosts/len(val_loader), 0]], columns = Results._fields)
            results = results.append(holding)
            print('Epoch' + str(epoch))
            print('Train Loss:', traincosts/len(train_loader), 'Val Loss:', valcosts/len(val_loader))
            print()
    except FileNotFoundError:
        print(dict_name, 'Not Found')
        
statspath = 'jgo_files/' + folder + '/' + folder + '_recon_stats.hdf5'      
results.to_hdf(statspath, 'results')
print('Model Statistics Saved to', statspath)

readmepath = 'jgo_files/' + folder + '/' + folder + '_recon_README.txt'
readme = open(readmepath, 'w')
readme.write('Loss Information Reconstructed from model information using Incomplete_Run_Loss_Reconstruction \n')
readme.write('\n')
readme.write('Model did not complete in desired run, so losses were reevaluated using recorded models \n')
readme.write('after the fact. There will not be time information in this file or in the stats file. ')
readme.write('\n')
readme.write('Model Architecture:' + ' \n')
childindex = 0
for child in model.children():
    readme.write(str(childindex) + '    ' + str(child) + ' \n')
    childindex = childindex+1
readme.write('\n')
readme.write('No Layers Frozen' + ' \n')
readme.write('\n')
readme.write('Pre-training Used: ' + previous + ' \n')
readme.write('\n')

readme.write('Validation Data Used' + ' \n')
for testfile in ['dataAA/20K_POCA_kernel_evts_200926.h5']:
    readme.write(testfile + '\n')
readme.write('No Validation Slice Used' + ' \n')
readme.write('\n')
readme.write('Batch Size:' + str(batch_size) + ' \n')
readme.write('\n')
readme.write('Loss Function: Ba' + ' \n')
readme.write('\n')
readme.write('Optimizer Used: Adam' + ' \n')
readme.write('\n')
readme.write('Latent Channels' + str(latentChannels) + ' \n')
readme.close()
print('Model Settings Saved to', readmepath)
        

Currently Analyzing jgo_files/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06_41.pyt
Starting Test Set
Completed in 15.240237474441528 s
Starting Train Set
Completed in 276.67628049850464 s
Epoch0
Train Loss: 2.814268037056923 Val Loss: 2.7428917138340374

Model Statistics Saved to jgo_files/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06_recon_stats.hdf5
Model Settings Saved to jgo_files/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06/11August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_exp_trainset_slow_finisher_128_epochs_1e-06_recon_README.txt


/usr/local/anaconda3/envs/july-2021-gpu/lib/python3.8/site-packages/pandas/core/generic.py:2700: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block1_values] [items->Index(['epoch', 'time'], dtype='object')]

  pytables.to_hdf(


In [ ]:
results = pd.DataFrame([], columns=Results._fields)
holding = pd.DataFrame([[epoch, traincosts/len(train_loader), valcosts/len(val_loader), 0]], columns = Results._fields)
results = results.append(holding)
print(results)

In [ ]:
time.time() - start


In [ ]:
quit()


In [ ]:
jgo_files/30July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_sliceless_250_epochs_1e-05/
30July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_sliceless_250_epochs_1e-05_recon_stats.hdf5
jgo_files/30July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_sliceless_250_epochs_1e-05/
30July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to50_nodes_sliceless_250_epochs_1e-05recon_stats.hdf5